# About:

Prepare the data to populate the 'TexteFiscExactCode' column from the 'Textes' table.

In [7]:
%reload_ext autoreload
%autoreload 2

import os, sys

sys.path.append(os.path.abspath(os.pardir))

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [9]:
COUNTRY_NAME = os.environ.get("COUNTRY_NAME")
SPOKEN_LANGUAGE = "French"

table_column_name = "TexteCode"

In [10]:
from general_config import COUNTRY_NAMES_LIST

# validate COUNTRY_NAME
if COUNTRY_NAME in COUNTRY_NAMES_LIST:
    print(COUNTRY_NAME, 'country name OK')

In [4]:
from llm_setup import *

# Get all documents for the selected country

In [5]:
(df, documents) = get_country_legal_docs(COUNTRY_NAME)

/home/andrei/Ferdi_LLM/postgres_connection.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


# Get all documents for the selected country

In [6]:
embedding_model = get_voyage_ai_embedding_model()

vector_index = get_vector_db_from_documents(documents, COUNTRY_NAME, embedding_model, chunk_size=1024)

loading from local


In [7]:
doc_titles = pd.Series(list(set([v.get('title') for k,v in vector_index.vector_store.to_dict()['metadata_dict'].items()])))
doc_titles[0]

'Décret n°1996-179 (19.06.1996) Application Office malien de l_habitat 1996 (SGG)'

# Get document year and number

In [8]:
import re

# Text to search
text = "img_Loi n°2006-017 (17.10.2006) Code minier 2006 (Droit-Afrique)"


def detect_doc_year_and_number(title):
    # Regex pattern to match "2006-017"
    pattern = r"\b\d{4}-\d{3}\b"

    # Search for the pattern
    match = re.search(pattern, title)

    # Return the matched string if found
    found_string = match.group(0) if match else None
    return found_string

In [9]:
pd.DataFrame(doc_titles.apply(detect_doc_year_and_number), columns=['DocYearAndNumber'])

,DocYearAndNumber
0,1996-179
1,2014-056
2,2016-520
3,2020-013
4,2012-063
...,...
70,2008-009
71,2011-078
72,2012-311
73,1991-065


In [10]:
df_TexteFiscExactCode = pd.DataFrame(doc_titles.apply(detect_doc_year_and_number), columns=['DocYearAndNumber'])

df_TexteFiscExactCode.to_csv(f"output/DocYearAndNumber_{COUNTRY_NAME}.csv", index=True)

df_TexteFiscExactCode

,DocYearAndNumber
0,1996-179
1,2014-056
2,2016-520
3,2020-013
4,2012-063
...,...
70,2008-009
71,2011-078
72,2012-311
73,1991-065


## Load existing categories

In [100]:
# COUNTRY_NAME = 'BEN Bénin'

large_categs = pd.read_csv(f"output/{table_column_name}_{COUNTRY_NAME}.csv")
large_categs = list(list(large_categs.set_index(['Unnamed: 0']).to_dict().values())[0].items())

## Update column in table

In [55]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import sessionmaker, declarative_base
# import logging

from postgres_connection import create_psql_engine

In [81]:

# # Configure logging
# logging.basicConfig(
#     level=logging.INFO,
#     filename='sqlalchemy_bulk_update.log',
#     filemode='a',
#     format='%(asctime)s - %(levelname)s - %(message)s'
# )

engine = create_psql_engine(db="Ferdi")

# Create a configured "Session" class
Session = sessionmaker(bind=engine)

# Create a Session
session = Session()

# Declare a mapping
Base = declarative_base()

class Textes(Base):
    __tablename__ = 'textes'
    
    TexteCode = Column("TexteCode", Integer, primary_key=True)
    PaysCode = Column("PaysCode", String, nullable=False)
    AnneeCodeDebut = Column("AnnéeCodeDébut", Integer)
    AnneeCodeFin = Column("AnnéeCodeFin", Integer)
    TexteLegalExactCode = Column("TexteLégalExactCode", String)
    TexteFiscExactCode = Column("TexteFiscExactCode", String)
    TexteCodeArborescence = Column("TexteCodeArborescence", String)
    TexteCourt = Column("TexteCourt", String)
    TexteComplet = Column("TexteComplet", String)

texts_table = Textes()

def bulk_update_textes(textes_to_update, column_name, texts_table=texts_table, COUNTRY_NAME=COUNTRY_NAME):
    """    
    :param employees_to_update: list of tuples (employee_id, new_annee_code_debut)
    :param column_name: valid column name from the textes table
    """
    for texte_code, new_col_value in textes_to_update:
        try:
            # Query the employee
            text_title = session.query(Textes).filter_by(TexteCode=texte_code, PaysCode=COUNTRY_NAME.split(" ")[0]).one_or_none()
            if text_title:
                # text_title.AnneeCodeDebut = new_code
                setattr(text_title, column_name, f"_{new_col_value}")
                session.commit()
                msg_ = f"{texte_code} column {column_name} updated to {new_col_value}."
                print(msg_)
                # logging.info(f"{texte_code} column {column_name} updated to {new_col_value}.")
            else:
                msg_ = f"{texte_code} not found."
                print(msg_)
                # logging.warning(f"{texte_code} not found.")
        except Exception as e:
            session.rollback()
            msg_ = f"Error updating {texte_code}: {e}"
            print(msg_)
            # logging.error(f"Error updating {texte_code}: {e}")


In [102]:
# bulk_update_employees(textes_to_update = large_categs[0:1], )

bulk_update_textes(
    textes_to_update = large_categs, 
    column_name = table_column_name, 
    texts_table=texts_table, 
    COUNTRY_NAME=COUNTRY_NAME)

# Close the session
session.close()

img_Circulaire n°2018-176 (05.03.2018) Application Loi de finances 2018 (DGI) column TexteFiscExactCode updated to Gen_LF_LFappli.
img_Loi n°2020-030 (28.10.2020) Loi de finances rectificative 2020 (SGG) column TexteFiscExactCode updated to Gen_LF_LFR.
Loi n°2007-033 (02.01.2008) Loi de finances 2008 (Droit-Afrique) column TexteFiscExactCode updated to Gen_LF_LF.
Loi n°2010-046 (30.12.2010) Loi de finances 2011 (Droit-Afrique) column TexteFiscExactCode updated to Gen_LF_LF.
img_Circulaire n°2017-118 (02.02.2017) Application Loi de finances 2017 (DGI) column TexteFiscExactCode updated to Gen_LF_LFappli.
Error updating img_Loi n°1997-014 (06.06.1997) Taxe sur les nuitées 1997 (LégiBénin): (psycopg2.errors.ForeignKeyViolation) insert or update on table "textes" violates foreign key constraint "textes_TexteFiscExactCode_fkey"
DETAIL:  Key (TexteFiscExactCode)=(_Gen_GST) is not present in table "textes_fiscaux_exacts".

[SQL: UPDATE textes SET "TexteFiscExactCode"=%(TexteFiscExactCode)s WHE